In [1]:
import pandas as pd

In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [3]:
train['num_ings'] = train['ingredients'].apply(lambda x : len(x))
test['num_ings'] = test['ingredients'].apply(lambda x : len(x))

In [4]:
train.head()

,cuisine,id,ingredients,num_ings
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,indian,22213,"[water, vegetable oil, wheat, salt]",4
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20


In [5]:
len(train)

39774

In [6]:
train = train[train['num_ings'] > 2]
len(train)

39559

In [7]:
from nltk.stem import WordNetLemmatizer

In [8]:
def get_replacements():
    return {'wasabe': 'wasabi', '-': '', 'sauc': 'sauce',
            'baby spinach': 'babyspinach', 'coconut cream': 'coconutcream',
            'coriander seeds': 'corianderseeds', 'corn tortillas': 'corntortillas',
            'cream cheese': 'creamcheese', 'fish sauce': 'fishsauce',
            'purple onion': 'purpleonion','refried beans': 'refriedbeans', 
            'rice cakes': 'ricecakes', 'rice syrup': 'ricesyrup', 
            'sour cream': 'sourcream', 'toasted sesame seeds': 'toastedsesameseeds', 
            'toasted sesame oil': 'toastedsesameoil', 'yellow onion': 'yellowonion'}

In [9]:
lemmatizer = WordNetLemmatizer()
replacements = get_replacements()

In [10]:
train['ingredients'] = train['ingredients'].apply(lambda x: list(map(lambda y: y.lower(), x)))
train.head()

,cuisine,id,ingredients,num_ings
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,indian,22213,"[water, vegetable oil, wheat, salt]",4
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20


In [11]:
test['ingredients'] = test['ingredients'].apply(lambda x: list(map(lambda y: y.lower(), x)))

In [12]:
import re
stop_pattern = re.compile('[\d’%]')

In [13]:
def tranform_to_single_string(ingredients, lemmatizer, replacements, stop_pattern):
    ingredients_text = ' '.join(iter(ingredients))

    for key, value in replacements.items():
        ingredients_text = ingredients_text.replace(key, value)
    
    words = []
    for word in ingredients_text.split():
        if not stop_pattern.match(word) and len(word) > 2: 
            word = lemmatizer.lemmatize(word)
            words.append(word)
    return ' '.join(words)

In [14]:
transform = lambda ingredients: tranform_to_single_string(ingredients, lemmatizer, replacements, stop_pattern)
train['x'] = train['ingredients'].apply(transform)
train.head()

,cuisine,id,ingredients,num_ings,x
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9,romaine lettuce black olive grape tomato garli...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11,plain flour ground pepper salt tomato ground b...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12,egg pepper salt mayonaise cooking oil green ch...
3,indian,22213,"[water, vegetable oil, wheat, salt]",4,water vegetable oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20,black pepper shallot cornflour cayenne pepper ...


In [15]:
test['x'] = test['ingredients'].apply(transform)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [17]:
vectorizer = make_pipeline(
        TfidfVectorizer(sublinear_tf=True),
        FunctionTransformer(lambda x: x.astype('float'), validate=False)
    )

In [18]:
train['x'] = train['ingredients'].apply(lambda x:  ' '.join(x))
test['x'] = test['ingredients'].apply(lambda x: ' '.join(x))
train.head()

,cuisine,id,ingredients,num_ings,x
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9,romaine lettuce black olives grape tomatoes ga...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11,plain flour ground pepper salt tomatoes ground...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12,eggs pepper salt mayonaise cooking oil green c...
3,indian,22213,"[water, vegetable oil, wheat, salt]",4,water vegetable oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20,black pepper shallots cornflour cayenne pepper...


In [23]:
len(train['x'].values[0].split())

16

In [25]:
x_train = vectorizer.fit_transform(train['x'].values)
x_train.sort_indices()
x_test = vectorizer.transform(test['x'].values)
x_train

<39559x3039 sparse matrix of type '<class 'numpy.float64'>'
	with 761223 stored elements in Compressed Sparse Row format>

In [32]:
print(x_train[0])

  (0, 185)	0.20746322346708837
  (0, 251)	0.13974375394788785
  (0, 528)	0.1456029044817406
  (0, 745)	0.33438840682765214
  (0, 967)	0.304068873691327
  (0, 1097)	0.38866212712136533
  (0, 1103)	0.10512521249020922
  (0, 1180)	0.3503982399997007
  (0, 1541)	0.26637601714980486
  (0, 1884)	0.26099187750248165
  (0, 1888)	0.16441971667203834
  (0, 2016)	0.10188799724128857
  (0, 2201)	0.23908938500384475
  (0, 2317)	0.342744772460344
  (0, 2426)	0.23005238144080467
  (0, 2797)	0.15171932869714977


In [37]:
def get_estimator():
    return SVC(C=300,
         kernel='rbf',
         gamma=1.5, 
         shrinking=True, 
         tol=0.001, 
         cache_size=1000,
         class_weight=None,
         max_iter=-1, 
         decision_function_shape='ovr',
         random_state=42)

In [38]:
estimator = get_estimator()

In [39]:
y_train = train['cuisine'].values
classifier = OneVsRestClassifier(estimator, n_jobs=-1)
classifier.fit(x_train, y_train)

OneVsRestClassifier(estimator=SVC(C=300, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.5, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1)

In [40]:
test['cuisine']  = classifier.predict(x_test)
test[['id', 'cuisine']].to_csv('submission.csv', index=False)